## Langchain demo of simple sequential chains

In class, Alex showed y'all basic simple sequential chains, we will buid on top of this today.
First, let's get all the config and imports stuff in order.

1. Notice the the below code block has `import config`, find the `config.py` file (in the same directory) and put your openai API key in here.  If you do not have one yet, create a new account and refer to [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key) to find your openai API key. 

2. Once you have your key, update the key in `config.py`, it should have the format of `sk-XXXXXXXXXXXXXXXXXX` (number of X's is not exact so don't take that literally please).

3. Run the block below once the above 2 are done so setup the environment.

In [1]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import HuggingFaceHub, LLMChain
from langchain.chains import SequentialChain, SimpleSequentialChain

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
import config
import os

os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

# uncomment below if you want to use huggingface instead, when you run this, you will see an input box below, paste in your huggingface token in there
# and wait to see "........"
# from getpass import getpass
# HUGGINGFACEHUB_API_TOKEN = getpass()
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

### What do we know?

We can build a chain by piping the output from 1 prompt into the input of another.  We saw this in class with poems and analysis.

The poem template was as such:
```
poem_template = """
You are a Poet. Given the title of a poem, write a haiku for that title.
Haiku for: 
{title}
"""

and analysis template as such:
```
analysis_template = """You are a poet critic. Given a haiku, it is your job to write an analysis.
Poem analysis for:
{haiku}
"""


And we chained them together like this: `overall_chain = SimpleSequentialChain(chains=[poem_chain, analysis_chain], verbose=True)`

Now this is pretty cool, but there is something subtle, see if you can spot it.
Hint: it has something to do with the initial input variable: title, where did it go?``
```
o do so.

#### The problem

If you have not figured it out already, our initial `title` variable is missing from the output!
In fact, imagine a chain as such

foo -> bar -> baz

where foo takes input a
bar takes input b
baz takes input c

so it becomes foo(a) -> bar(b = foo(a)) -> baz(c = bar(foo(a)))

Since every chain effectively ***consumes*** the input, that input is lost forever (or is it??) from the final output.  If you run the chain discussed in class, all we get is the analysis of the poem, we miss the poem title and the poem itself.

We will recreate the situation below with food and restaurants.

In [2]:
# Set up the model with temperature
# temperature is a measure of how creative we want the model to be
temperature = 0.5
model = OpenAI(temperature=temperature)

# uncomment below to test you have credits, if not go back up and use huggingface
name = model("say hi")

# uncomment below to use huggingface
# repo_id = "google/flan-t5-xxl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
# hmodel = HuggingFaceHub(
#     repo_id=repo_id, model_kwargs={"temperature": temperature, "max_length": 64}
# )

In [3]:
prompt_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "Suggest a fancy name for a restaurant that serves {cuisine} food."
)

# change to hmodel if you want to use huggingface
name_chain = LLMChain(llm=model, prompt=prompt_name, output_key="restaurant_name")

prompt_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}"
)

menu_items_chain = LLMChain(llm=model, prompt=prompt_items, output_key="menu_items")

overall_chain = SimpleSequentialChain(chains=[name_chain, menu_items_chain], verbose=True)
print(overall_chain.run('Mexican'))



> Entering new SimpleSequentialChain chain...


Taco Tequila Palace.
 

-Taco Salad 
-Beef Fajitas 
-Chicken Quesadillas 
-Chimichangas 
-Taco Platter 
-Grilled Fish Tacos 
-Nacho Supreme 
-Chili con Carne 
-Enchiladas 
-Tequila Lime Chicken 
-Taco Pizza 
-Taco Rice Bowl 
-Taco Soup 
-Tequila Shrimp 
-Tequila Margarita 
-Fried Ice Cream

> Finished chain.
 

-Taco Salad 
-Beef Fajitas 
-Chicken Quesadillas 
-Chimichangas 
-Taco Platter 
-Grilled Fish Tacos 
-Nacho Supreme 
-Chili con Carne 
-Enchiladas 
-Tequila Lime Chicken 
-Taco Pizza 
-Taco Rice Bowl 
-Taco Soup 
-Tequila Shrimp 
-Tequila Margarita 
-Fried Ice Cream


If you run the above, notice the last line is just the menu items without the restaurant name!

#### Solution

In comes SequentialChains, these are more verbose than SimpleSequentialChains and include more information that we can use in other parts of our code

In [4]:
seq_chain = SequentialChain(
    chains = [name_chain, menu_items_chain], 
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)
seq_chain({'cuisine': 'Chinese'})

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-vhyx88p93vYYVt9KRoCGdsWp on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-vhyx88p93vYYVt9KRoCGdsWp on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to a

{'cuisine': 'Chinese',
 'restaurant_name': '\n\nImperial Dragon Palace.',
 'menu_items': '\n\n- Crispy Aromatic Duck\n- Kung Pao Chicken\n- Sweet & Sour Pork\n- Hot & Sour Soup\n- Egg Fried Rice\n- Szechuan Beef\n- Stir-Fried Vegetables\n- Peking Style Noodles\n- Spring Rolls\n- Vegetarian Dumplings\n- Steamed Fish with Ginger and Spring Onion\n- Spicy Salt & Pepper Squid\n- Coconut Prawns'}

And you can see, running the above gives us the cuisine, restaurant_name and menu_items as intended!

### Exercise

Below, come up with another sequential chain with >= 3 nodes total, make it different than poems and food and show all output keys.

In [5]:
prompt_name = PromptTemplate(
    input_variables = ['character_type'],
    template = "Suggest a suitable name for a {character_type}"
)

# change to hmodel if you want to use huggingface
cname_chain = LLMChain(llm=model, prompt=prompt_name, output_key="character_name")

prompt_items = PromptTemplate(
    input_variables = ['character_name'],
    template = "Suggest a backstory for {character_name}"
)

story_chain = LLMChain(llm=model, prompt=prompt_items, output_key="story")

prompt_name = PromptTemplate(
    input_variables = ['story'],
    template = "Suggest names for a weapon that a character with the following backstory would use; {story}"
)

# change to hmodel if you want to use huggingface
weapon_chain = LLMChain(llm=model, prompt=prompt_name, output_key="weapon_names")

overall_chain = SequentialChain(chains=[cname_chain, story_chain, weapon_chain], input_variables=['character_type'], output_variables=['character_name', 'story', 'weapon_names'])
overall_chain({'character_type': 'Villain'})

{'character_type': 'Villain',
 'character_name': '\n\nDr. Doom',
 'story': "\n\nVictor von Doom was born into a Romani family in Latveria. His parents were a powerful couple, with his father being a renowned alchemist and his mother a powerful sorceress. From a young age, Victor was exposed to both the science and the mysticism of his parents, and developed an interest in both.\n\nAt the age of 16, Victor was sent to the United States to attend college. While there, Victor excelled in his studies, particularly in the fields of physics and engineering. He also studied the mystic arts under a powerful sorcerer, and was able to combine his knowledge of science and magic to create powerful inventions. \n\nHowever, Victor's arrogance and ambition led him to attempt a dangerous experiment to contact the spirit realm. The experiment went horribly wrong, leaving Victor's face disfigured and his body scarred.\n\nFilled with rage and a desire for revenge, Victor returned to Latveria and used his